In [1]:
from Neuroradiomics.registration import registration_reader
from Neuroradiomics.registration import elastix_multimap_registration
from Neuroradiomics.registration import registration_writer
from Neuroradiomics.registration import apply_transform_from_files
from Neuroradiomics.registration import read_transform_from_files
from Neuroradiomics.registration import elastix_rigid_registration
from Neuroradiomics.registration import Set_sampler_parameters_as_image
from Neuroradiomics.registration import registration_transform_parameters_writer
from Neuroradiomics.registration import transform_parameters_writer
from Neuroradiomics.registration import evaluate_registration_mse
#from Neuroradiomics.skull_stripping import masking



import itk
import numpy as np
import os
from itkwidgets import view

import tkinter as tk
from tkinter import filedialog

In [2]:
root = tk.Tk()
root.withdraw()


''

In [3]:
flair_filename = filedialog.askopenfilename(title = "Select the flair image")

    #Moving Image
t1_filename = filedialog.askopenfilename(title = "Select the t1 image")

atlas_filename = filedialog.askopenfilename(title = "Select the atlas image")

mask_filename = filedialog.askopenfilename(title = "Select the brain mask image")

In [4]:
flair, t1 = registration_reader(flair_filename, t1_filename)
atlas, mask = registration_reader(atlas_filename, mask_filename)


In [11]:
rigid_reg_obj = elastix_rigid_registration(flair, t1)
new_params = Set_sampler_parameters_as_image(rigid_reg_obj.GetTransformParameterObject(), t1)

The Rigid Registration is done!


In [12]:
reg_t1 = itk.transformix_filter(t1, new_params)                                

In [13]:
reg_atlas_obj = elastix_multimap_registration(reg_t1, atlas)

The Multimap Registration is done!


In [14]:
reg_mask = itk.transformix_filter(mask, reg_atlas_obj.GetTransformParameterObject())

In [26]:
def masking (image, mask, Dimension = 3):
    
    masked_image = itk.Image[itk.F, Dimension].New()
    
    masked_image.SetRegions(image.GetLargestPossibleRegion())
    masked_image.Allocate()
    
    index = itk.Index[Dimension]()
    
    for index[0] in range( mask.GetLargestPossibleRegion().GetSize()[0] ):
    
        for index[1] in range( mask.GetLargestPossibleRegion().GetSize()[1] ):
        
            for index[2] in range( mask.GetLargestPossibleRegion().GetSize()[2] ):
            
                if mask.GetPixel(index) < 0.9:
                    masked_image.SetPixel(index, 0)
                else: 
                    masked_image.SetPixel(index, image.GetPixel(index))
                
    
    return masked_image

In [27]:
masked_t1 = masking(reg_t1, reg_mask)

In [17]:
view(masked_t1)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [28]:
! cd /home/nicolas/Desktop/Neuroradiomics/Prova
itk.imwrite(masked_t1, '/home/nicolas/Desktop/Neuroradiomics/Prova/masked_t1_diocancan.nii')

In [20]:
!cd /

__init__.py.py	registration_console.ipynb  run_stripper.py
LICENSE		requirements.txt	    run_transformation.py
Neuroradiomics	run_registration.py	    Tests


In [ ]:
#view(t1)